In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

23/09/20 08:10:15 WARN Utils: Your hostname, elvin-Aspire-E1-571 resolves to a loopback address: 127.0.1.1; using 192.168.236.183 instead (on interface wlp3s0)
23/09/20 08:10:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/20 08:10:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/20 08:10:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/20 08:10:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Preparando dados para Classificação

In [2]:
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
churn = spark.read.csv('/home/elvin/download/Churn.csv', inferSchema=True, header=True, sep=';')
churn.show(5)

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+-------

In [4]:
formula = RFormula(formula="Exited ~ .", featuresCol="features", labelCol="label", handleInvalid="skip") 
# Variável dependente é Exited e as variáveis independentes são todas as demais (representados por '.')

In [5]:
churnTrans = formula.fit(churn).transform(churn).select("features", "label")

In [6]:
churnTrans.show(truncate=False)

+----------------------------------------------------------------+-----+
|features                                                        |label|
+----------------------------------------------------------------+-----+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]        |1.0  |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]  |0.0  |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]  |1.0  |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])          |0.0  |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0]   |0.0  |
|[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0  |
|[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0]           |0.0  |
|[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0  |
|[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0]   |0.0  |
|[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0]  |0.0  |
|[528.0,1.0,0.0,1.0,31.0,6.0,1.0201672E7,2.0,0.0,0.

# Criando um modelo de classificação

In [7]:
churnTreino, churnTeste = churnTrans.randomSplit([0.7,0.3])
print(churnTreino.count())
print(churnTeste.count())

7011
2989


In [8]:
dtclassifier = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [9]:
modelo = dtclassifier.fit(churnTreino)

In [10]:
previsao = modelo.transform(churnTeste)

In [11]:
previsao.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(11,[0,1,3,4,7,10...|  0.0| [115.0,115.0]|           [0.5,0.5]|       0.0|
|(11,[0,1,3,4,7,10...|  0.0|  [188.0,44.0]|[0.81034482758620...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4386.0,466.0]|[0.90395713107996...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4386.0,466.0]|[0.90395713107996...|       0.0|
|(11,[0,1,4,5,7,10...|  1.0|  [188.0,44.0]|[0.81034482758620...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4386.0,466.0]|[0.90395713107996...|       0.0|
|(11,[0,1,4,5,7,10...|  1.0|  [47.0,254.0]|[0.15614617940199...|       1.0|
|(11,[0,1,4,5,7,10...|  0.0|[4386.0,466.0]|[0.90395713107996...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4386.0,466.0]|[0.90395713107996...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4386.0,466.0]|[0.90395713107996...|       0.0|
|(11,[0,1,4,

In [12]:
avaliar = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")

In [13]:
areaUnderRoc = avaliar.evaluate(previsao)

In [14]:
print(areaUnderRoc)

0.6712570686937981
